In [3]:
import numpy as np
from numpy import random
import math

MAXG=5000
# define population size
PSIZE=50
D = 10
# define lower and upper bounds for every dimension
#bounds = np.array(limites(n,-5,5))#np.asarray([(-5.0, 5.0), (-5.0, 5.0)])

PM = 0.01
PC = 0.65

F = 0.5
Cr = 0.6

ACKLEY1 = 1
ACKLEY2 = 2
ACKLEY3 = 3
ACKLEY4 = 4

'''subject to −35 ≤ xi ≤ 35. The global minima is located at origin x∗ = (0, · · · , 0),
f(x∗) = 0.'''
def ackley_1(individuo):
    global D
    sum1 = 0.0
    sum2 = 0.0
    for g in individuo:
        sum1 = sum1 + g**2.0
        sum2 = sum2 + np.cos(2.0*np.pi*g)
    return -20.0*np.exp(-0.02*np.sqrt(sum1/D)) - np.exp(sum2/D) + 20 + np.e


'''subject to −32 ≤ xi ≤ 32. The global minimum is located at origin x∗ = (0, 0),
f(x∗) = −200. '''  
def ackley_2(individuo):
    return -200*np.exp(-0.02*np.sqrt(individuo[0]**2+individuo[1]**2))


'''subject to −32 ≤ xi ≤ 32. The global minimum is located at x∗ = (0, ≈ −0.4),
f(x∗) ≈ −219.1418.'''
def ackley_3(individuo):
    return ackley_2(individuo)+5*np.exp(np.cos(3*individuo[0])+np.sin(3*individuo[1]))


'''subject to −35 ≤ xi ≤ 35. It is highly multimodal function with two global minimum
close to origin
x = f({−1.479252, −0.739807}, {1.479252, −0.739807}), f(x∗) = −3.917275'''
def ackley_4(individuo):
    global D
    suma = 0.0
    i=0
    while (i+1 < D):
        suma = suma + np.exp(-0.2)*np.sqrt(individuo[i]**2+individuo[i+1]**2)
        +3*(np.cos(2*individuo[i])+np.sin(2*individuo[i+1]))
        i = i + 1
    return suma

def init_populations(pop_size = PSIZE):
    global D
    pop_1 = []
    pop_2 = []
    pop_3 = []
    pop_4 = []
    for i in range(pop_size):
        pop_1.append([random.uniform(-35, 35) for i in range(D)])
        pop_2.append([random.uniform(-32, 32), random.uniform(-32, 32)])
        pop_3.append([random.uniform(-32, 32), random.uniform(-32, 32)])
        pop_4.append([random.uniform(-35, 35) for i in range(D)])
    return np.array(pop_1),np.array(pop_2),np.array(pop_3),np.array(pop_4)

def calculate_fitness(pop,ackley):
    switch = {
        1: ackley_1,
        2: ackley_2,
        3: ackley_3,
        4: ackley_4
    }
    scores = []
    for ind in pop:
        scores.append(switch.get(ackley)(ind))
    return scores

# define crossover operation
def crossover(mutated, target, dims, cr):
    # generate a uniform random value for every dimension
    p = random.rand(dims)
    # generate trial vector by binomial crossover
    trial = [mutated[i] if p[i] < cr else target[i] for i in range(dims)]# la intercalación
    return trial

# define mutation operation
def mutation(x, F):
    return x[0] + F * (x[1] - x[2])

# define boundary check operation
def check_bounds(mutated, bounds):
    #Va de i hasta D                       tomando li & ls
    mutated_bound = [np.clip(mutated[i], bounds[i][0], bounds[i][1]) for i in range(len(bounds))]
    return mutated_bound

def differential_evolution(ackley,pop,bounds,scores,best_score,best_scores_progress):
    mejor_puntaje = float('inf')
    mejor_solucion = []
    obj_iter = list()
    
    # Init, find the best performing vector of initial population
    best_score=np.min(scores)
    best_scores_progress.append(best_score)
    
    for generation in range(MAXG):
        #new_population=[]
        if best_score < mejor_puntaje:
            mejor_puntaje = best_score
            mejor_solucion = pop[np.argmin(scores)][:]
        # iterate over all candidate solutions
        for i in range(int(PSIZE)):
            # choose three candidates, a, b and c, that are not the current one
            candidates = [candidate for candidate in range(PSIZE) if candidate != i]
            a, b, c = pop[random.choice(candidates, 3, replace=False)]
            # perform mutation
            mutated = mutation([a, b, c], F)
            # check that lower and upper bounds are retained after mutation
            mutated = check_bounds(mutated, bounds) #"bounds" = "D" = [ [li ls],[li ls],... ] 
            
            # perform crossover binomial
            trial = crossover(mutated, pop[i], len(bounds), Cr)
           
            # compute objective function value for target vector
            pop_arr = []
            pop_arr.append(pop[i])
            obj_target = calculate_fitness(pop_arr,ackley) #HAY QUE CASTEAR A ARREGLO PARA QUE SOLO ME DEVUELVA 1 RESULTADO
            #DEBERIA SER obj_target[0]
            # compute objective function value for trial vector
            pop_trial = []
            pop_trial.append(trial)
            obj_trial = calculate_fitness(pop_trial,ackley)
            #DEBERIA SER obj_trial[0]
            
            # perform selection
            if obj_trial[0] < obj_target[0]:
                # replace the target vector with the trial vector
                pop[i] = trial
                # store the new objective function value
                scores[i] = obj_trial[0]
                
        # find the best performing vector at each iteration
        best_score = np.min(scores)    
        best_scores_progress.append(best_score)
    return [best_scores_progress, best_score, scores]
 
        
bounds1 = [[-35, 35] for i in range(D)]
bounds2 = [-32, 32]
bounds3 = [[-32, 32] for i in range(D)]
bounds4 = [-35, 35]
    
pop1, pop2, pop3, pop4 = init_populations()
scores = []
best_score = 0
best_scores_progress = []
scores1 = calculate_fitness(pop1,ACKLEY1) 
# perform differential evolution
solution = differential_evolution(ACKLEY1,pop1,bounds1,scores1,best_score,best_scores_progress)
print('\nSolution: f([%s]) = %.5f' % (np.around(solution[0], decimals=5), solution[1]))


#print('\nSolution: f([%s]) = %.5f' % (around(solution[0], decimals=5), solution[1]))

'''

print(ejecucion_ga(pop1,ACKLEY1,scores,best_score,best_scores_progress))
print('---')
scores = calculate_fitness(pop2,ACKLEY2)
print(ejecucion_ga(pop2,ACKLEY2,scores,best_score,best_scores_progress))
print('---')
scores = calculate_fitness(pop3,ACKLEY3)
print(ejecucion_ga(pop3,ACKLEY3,scores,best_score,best_scores_progress))
print('---')
scores = calculate_fitness(pop4,ACKLEY4)
print(ejecucion_ga(pop4,ACKLEY4,scores,best_score,best_scores_progress))
'''




Solution: f([[6.01628 6.01628 5.42483 ... 0.      0.      0.     ]]) = 0.00000


"\n\nprint(ejecucion_ga(pop1,ACKLEY1,scores,best_score,best_scores_progress))\nprint('---')\nscores = calculate_fitness(pop2,ACKLEY2)\nprint(ejecucion_ga(pop2,ACKLEY2,scores,best_score,best_scores_progress))\nprint('---')\nscores = calculate_fitness(pop3,ACKLEY3)\nprint(ejecucion_ga(pop3,ACKLEY3,scores,best_score,best_scores_progress))\nprint('---')\nscores = calculate_fitness(pop4,ACKLEY4)\nprint(ejecucion_ga(pop4,ACKLEY4,scores,best_score,best_scores_progress))\n"